<a href="https://colab.research.google.com/github/yoooniverse/NLP/blob/main/NaiveBayesClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

02.22.2023

Bayes' theorem(조건부확률 계산 방법 중 하나)  

$P(A|B) = {P(B|A)P(A)\over P(B)}$

P(정상 메일 | 입력 텍스트) = 입력 텍스트가 있을 때 정상 메일일 확률  
P(스팸 메일 | 입력 텍스트) = 입력 텍스트가 있을 때 스팸 메일일 확률



P(정상 메일 | 입력 텍스트) = (P(입력 텍스트 | 정상 메일) × P(정상 메일)) / P(입력 텍스트)  
P(스팸 메일 | 입력 텍스트) = (P(입력 텍스트 | 스팸 메일) × P(스팸 메일)) / P(입력 텍스트)

분모를 양쪽에서 제거하여 식을 간소화.  
P(정상 메일 | 입력 텍스트) = P(입력 텍스트 | 정상 메일) × P(정상 메일)  
P(스팸 메일 | 입력 텍스트) = P(입력 텍스트 | 스팸 메일) × P(스팸 메일)

P(정상 메일 | 입력 텍스트)가 P(스팸 메일 | 입력 텍스트)보다 크다면 정상 메일이라고 볼 수 있다.


입력 텍스트에 단어가 3개 있다면,

$P(정상 메일 | 입력 텍스트) = P(w_1 | 정상 메일) × P(w_2 | 정상 메일) × P(w_3 | 정상 메일) × P(정상 메일)$

$P(스팸 메일 | 입력 텍스트) = P(w_1 | 스팸 메일) × P(w_2 | 스팸 메일) × P(w_3 | 스팸 메일) × P(스팸 메일)$

point) 입력 텍스트에 대해서 단, 하나의 단어라도 훈련 텍스트에 없었다면 확률 전체가 0이 되는 것은 지나친 일반화

**라플라스 스무딩) 각 단어에 대한 확률의 분모, 분자에 전부 숫자를 더해서 분자가 0이 되는 것을 방지**

In [1]:
from sklearn.datasets import fetch_20newsgroups
newsdata = fetch_20newsgroups(subset='train')
print(newsdata.keys())

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])


In [2]:
print(len(newsdata.data), len(newsdata.filenames), len(newsdata.target_names), len(newsdata.target))

11314 11314 20 11314


In [3]:
print(newsdata.target_names)

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [4]:
print(newsdata.target)

[7 4 4 ... 3 1 8]


In [5]:
print(newsdata.target[0])
print(newsdata.target_names[7])
print(newsdata.data[0])

7
rec.autos
From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
Organization: University of Maryland, College Park
Lines: 15

 I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.

Thanks,
- IL
   ---- brought to you by your neighborhood Lerxst ----







newsdata.target & newsdata.data

need to preprocess `newsdata.data`

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

In [7]:
# 나이브 베이즈 분류를 위해서는 데이터를 BoW로 만들어줄 필요
# 입력한 텍스트를 자동으로 BoW로 만드는 CountVectorizer

dtmvector = CountVectorizer()
X_train_dtm = dtmvector.fit_transform(newsdata.data)
print(X_train_dtm.shape)

(11314, 130107)


BoW의 원리(Bag of Words)

(1) 각 단어에 고유한 정수 인덱스를 부여.  # 단어 집합 생성.  
(2) 각 인덱스의 위치에 단어 토큰의 등장 횟수를 기록한 벡터를 생성.  
사이킷 런에서는 단어의 빈도를 Count하여 Vector로 만드는 CountVectorizer 클래스를 지원. 이를 이용하면 영어에 대해서는 손쉽게 BoW를 만들 수 있다.

DTM (Document-Term Matrix)

서로 다른 문서들의 BoW들을 결합한 표현 방법

TF-IDF(Term Frequency-Inverse Document Frequency)

DTM 내의 각 단어들마다 중요한 정도를 가중치로 주는 방법 (사이킷 런 사용)

In [8]:
tfidf_transformer = TfidfTransformer()
tfidfv = tfidf_transformer.fit_transform(X_train_dtm)
print(tfidfv.shape)

(11314, 130107)


In [9]:
mod = MultinomialNB()
mod.fit(tfidfv, newsdata.target)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True) #alpha 1.0 : 라플라스 스무딩

MultinomialNB()

In [10]:
newsdata_test = fetch_20newsgroups(subset='test', shuffle=True)
X_test_dtm = dtmvector.transform(newsdata_test.data)
tfidfv_test = tfidf_transformer.transform(X_test_dtm)

predicted = mod.predict(tfidfv_test)
print("정확도: ", accuracy_score(newsdata_test.target, predicted))

정확도:  0.7738980350504514


note) 잠재 의미 분석 챕터에서 진행했던 전처리를 모두 진행하고 다시 나이브 베이즈 분류기를 돌려보세요. 80% 이상의 정확도를 얻을 수 있습니다

https://wikidocs.net/24949